# Hedge Fund X
This notebook solve the Hedge Fund X's competition challenge: Financial Modeling challenge.
This one is used to evaluate model, tuning param to find most sutable model.
To discover data set, check the other Notebook.

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import Imputer, StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn import metrics
from xgboost import XGBClassifier

from sklearn.externals import joblib
import datetime

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df = pd.read_csv("../input/hedge_fund_x/train.csv")
df_test = pd.read_csv("../input/hedge_fund_x/test.csv")
df.head(5)

,data_id,period,c1,c2,c3,c4,c5,c6,c7,c8,...,c80,c81,c82,c83,c84,c85,c86,c87,c88,target
0,2,train1,0.655570,-0.000022,-0.000539,-0.001075,0.0,0.0,0.213390,0.0,...,-0.023358,-0.017041,0.0,0.060697,0.0,0.0,0.000000,-0.000202,-0.140220,1
1,3,train1,1.646430,-0.000292,-0.008367,0.009497,0.0,0.0,0.000000,0.0,...,-0.059429,-0.009109,0.0,0.021645,0.0,0.0,0.000000,-0.004382,0.455767,0
2,5,train1,-0.743010,0.004642,-0.000647,-0.003290,0.0,0.0,0.000000,0.0,...,0.001796,-0.000104,0.0,-0.024718,0.0,0.0,0.219566,0.072711,1.155580,0
3,7,train1,0.029770,-0.006343,-0.000635,-0.002516,0.0,0.0,0.160313,0.0,...,-0.005501,0.045308,0.0,-0.148852,0.0,0.0,0.000000,-0.101181,-0.954553,0
4,10,train1,-0.660243,0.012591,-0.002098,-0.022264,0.0,0.0,0.000000,0.0,...,0.029034,-0.005847,0.0,-0.007073,0.0,0.0,0.000000,-0.004842,0.436002,0


In [46]:
clf_xgb_period = joblib.load('clf_xgb_period_40000_14.pkl')
clf_xgb = joblib.load('clf_xgb_40000_14.pkl')

In [47]:
def print_val_score(scores, label):
    print "{}: {:.2f} (+/- {:.2f})".format(
        label,
        scores[label].mean(), 
        scores[label].std())
def evaluate_model(est, train_df, excluded_cols = ['period', 'target']):
    selected_cols = [col for col in train_df.columns if col not in excluded_cols]
    X_train = train_df[selected_cols].values
    y_train = train_df['target'].values

    scoring={'acc':'accuracy', 'log_loss':'neg_log_loss'}
    scores = cross_validate(estimator=est, X=X_train, y=y_train, cv=10, scoring=scoring, 
                            return_train_score=True)
    print_val_score(scores, 'train_acc')
    print_val_score(scores, 'test_acc')
    print_val_score(scores, 'train_log_loss')
    print_val_score(scores, 'test_log_loss')

In [8]:
def evaluate_period(est, train_df, excluded_cols = ['period', 'target']):
    selected_cols = [col for col in train_df.columns if col not in excluded_cols]
    X_test = train_df[selected_cols].values
    y_test = train_df['target'].values
    y_pred = est.predict(X_test)
    y_pred_proba = clf_xgb.predict_proba(X_test)[:,1]
    return y_pred, y_pred_proba, y_test

In [48]:
def print_pred_score(y_pred, y_pred_proba, y_test):
    print "Accuracy scrore: {}".format(metrics.accuracy_score(y_test, y_pred))
    print "Log loss scrore: {}".format(metrics.log_loss(y_test, y_pred_proba))

## Remove noise
There are some column that has so much noise, going to train with those collumn removed

In [49]:
zero_count = (train1_df==0.0).sum(axis=0).sort_values(ascending=False)
has_zero_ratio = zero_count[zero_count > 0].div(train1_df.shape[0])
excluded_cols = has_zero_ratio.index.values
excluded_cols_period = [x for x in excluded_cols]
excluded_cols_period.append('period')
excluded_cols_period.append('data_id')
print excluded_cols_period

['c45', 'c34', 'c8', 'c37', 'c23', 'c18', 'c6', 'c28', 'c62', 'c85', 'c46', 'c60', 'c5', 'c32', 'c55', 'c65', 'c41', 'c54', 'c86', 'c70', 'c63', 'c82', 'c7', 'c58', 'c52', 'c19', 'c10', 'c17', 'c13', 'c84', 'c75', 'c42', 'target', 'period', 'data_id']


In [50]:
def evaluate_period_prediction(est, train_df, excluded_cols = ['period', 'target']):
    selected_cols = [col for col in train_df.columns if col not in excluded_cols]
    X_test = train_df[selected_cols].values
    y_test = train_df['period'].values
    y_pred = est.predict(X_test)
    return y_test, y_pred

In [51]:
train1_df = df[df.period == 'train1']
train2_df = df[df.period == 'train2']
train3_df = df[df.period == 'train3']
train4_df = df[df.period == 'train4']
train5_df = df[df.period == 'train5']
train6_df = df[df.period == 'train6']
train7_df = df[df.period == 'train7']

In [52]:
print metrics.accuracy_score(*evaluate_period_prediction(clf_xgb_period, train1_df, excluded_cols_period))
print metrics.accuracy_score(*evaluate_period_prediction(clf_xgb_period, train2_df, excluded_cols_period))
print metrics.accuracy_score(*evaluate_period_prediction(clf_xgb_period, train3_df, excluded_cols_period))
print metrics.accuracy_score(*evaluate_period_prediction(clf_xgb_period, train4_df, excluded_cols_period))
print metrics.accuracy_score(*evaluate_period_prediction(clf_xgb_period, train5_df, excluded_cols_period))

0.9658
0.9613
0.98565
0.978625
0.974875


In [55]:
selected_cols = [col for col in df_test.columns if col not in excluded_cols_period]
X_test = df_test[selected_cols].values

In [56]:
X_test_period = clf_xgb_period.predict(X_test)
print X_test_period

['train8' 'train2' 'train1' ..., 'train11' 'train14' 'train2']


In [57]:
print pd.DataFrame(X_test_period).describe()

             0
count   361500
unique      14
top     train6
freq     45251


In [58]:
train1_no_period_df = train1_df.drop(['period'], axis=1)[selected_cols].values
train2_no_period_df = train2_df.drop(['period'], axis=1)[selected_cols].values
train3_no_period_df = train3_df.drop(['period'], axis=1)[selected_cols].values
train4_no_period_df = train4_df.drop(['period'], axis=1)[selected_cols].values
print train4_df.drop(['period'], axis=1)[selected_cols].describe()

                 c1            c2            c3            c4            c9  \
count  40000.000000  40000.000000  40000.000000  40000.000000  40000.000000   
mean       0.194275      0.000022      0.001134      0.001854     -0.002790   
std        0.993199      0.011474      0.016211      0.049617      0.017819   
min       -3.170218     -0.101758     -0.096984     -0.342562     -0.212850   
25%       -0.547481     -0.003520     -0.005499     -0.019518     -0.009165   
50%        0.266653      0.000021      0.000312      0.000530     -0.000997   
75%        0.986977      0.003843      0.006674      0.022055      0.004756   
max        2.813221      0.079254      0.301534      1.025511      0.134326   

                c11           c12           c14           c15           c16  \
count  40000.000000  40000.000000  40000.000000  40000.000000  40000.000000   
mean      -0.011353     -0.025205      0.001180     -0.001357      0.001915   
std        0.054041      0.069511      0.035470    

In [59]:
train1_period_pred = clf_xgb_period.predict(train1_no_period_df)
print pd.DataFrame(train1_period_pred).describe()
train2_period_pred = clf_xgb_period.predict(train2_no_period_df)
print pd.DataFrame(train2_period_pred).describe()
train3_period_pred = clf_xgb_period.predict(train3_no_period_df)
print pd.DataFrame(train3_period_pred).describe()
train4_period_pred = clf_xgb_period.predict(train4_no_period_df)
print pd.DataFrame(train4_period_pred).describe()

             0
count    40000
unique      13
top     train1
freq     38632
             0
count    40000
unique      14
top     train2
freq     38452
             0
count    40000
unique      13
top     train3
freq     39426
             0
count    40000
unique      12
top     train4
freq     39145


### Predict period for data

In [289]:
oh = OneHotEncoder()
le = LabelEncoder()
period_values = df['period']
le_period = le.fit_transform(period_values)
print le_period[90000:90005]

[7 7 7 7 7]


In [291]:
oh_period = oh.fit_transform(le_period.reshape(-1,1)).toarray()
print oh_period

[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [293]:
print oh_period.shape

(560000, 14)


In [297]:
oh_period_df = pd.DataFrame(oh_period)
print oh_period_df.head(5)

    0    1    2    3    4    5    6    7    8    9    10   11   12   13
0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
4  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0


In [299]:
enc_df = pd.concat([df, oh_period_df], axis=1)
print enc_df.head(5)

   data_id  period        c1        c2        c3        c4   c5   c6  \
0        2  train1  0.655570 -0.000022 -0.000539 -0.001075  0.0  0.0   
1        3  train1  1.646430 -0.000292 -0.008367  0.009497  0.0  0.0   
2        5  train1 -0.743010  0.004642 -0.000647 -0.003290  0.0  0.0   
3        7  train1  0.029770 -0.006343 -0.000635 -0.002516  0.0  0.0   
4       10  train1 -0.660243  0.012591 -0.002098 -0.022264  0.0  0.0   

         c7   c8 ...     4    5    6    7    8    9   10   11   12   13  
0  0.213390  0.0 ...   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
1  0.000000  0.0 ...   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2  0.000000  0.0 ...   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3  0.160313  0.0 ...   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
4  0.000000  0.0 ...   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 105 columns]


In [300]:
def print_val_score(scores, label):
    print "{}: {:.2f} (+/- {:.2f})".format(
        label,
        scores[label].mean(), 
        scores[label].std())
def evaluate_model(est, train_df, excluded_cols = ['period', 'target']):
    selected_cols = [col for col in train_df.columns if col not in excluded_cols]
    X_train = train_df[selected_cols].values
    y_train = train_df['target'].values

    scoring={'acc':'accuracy', 'log_loss':'neg_log_loss'}
    scores = cross_validate(estimator=est, X=X_train, y=y_train, cv=10, scoring=scoring, 
                            return_train_score=True)
    print_val_score(scores, 'train_acc')
    print_val_score(scores, 'test_acc')
    print_val_score(scores, 'train_log_loss')
    print_val_score(scores, 'test_log_loss')

In [303]:
train_df = pd.concat([enc_df[x*40000: x*40000 + 1000] for x in range(0,14)])
print train_df.shape

(14000, 105)


In [307]:
selected_cols = [col for col in train_df.columns if col not in excluded_cols_period]
X_train = train_df[selected_cols].values
y_train = train_df['target'].values
print X_train.shape

(14000, 71)


In [310]:
clf_xgb = XGBClassifier(max_depth=7, n_estimators=150, reg_lambda=100)
clf_xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=7,
       min_child_weight=1, missing=None, n_estimators=150, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=100,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [314]:
enc_train_1000_9000_df = pd.concat([enc_df[x*40000 + 1000: x*40000 + 9000] for x in range(0,14)])
print_pred_score(*evaluate_period(enc_train_1000_2000_df, clf_xgb, excluded_cols_period))

Accuracy scrore: 0.5825
Log loss scrore: 0.670210687175


## Try to predict test

In [325]:
selected_cols = [col for col in df_test.columns if col not in excluded_cols]
X_test = df_test[selected_cols].values

In [326]:
X_test_period = clf_xgb_period.predict(X_test)
print X_test_period

['train14' 'train14' 'train14' ..., 'train14' 'train14' 'train14']


In [332]:
X_test_period_enc = oh.transform(le.transform(X_test_period).reshape(-1,1)).toarray()
print X_test_period_enc

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [352]:
print X_test_period_enc.shape

(361500, 14)


In [354]:
X_test_period_enc_df = pd.DataFrame(X_test_period_enc)
print X_test_period_enc_df.head(5)
print X_test_period_enc_df.shape

    0    1    2    3    4    5    6    7    8    9    10   11   12   13
0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
4  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
(361500, 14)


In [355]:
X_test_enc_df = pd.concat([df_test, X_test_period_enc_df], axis=1)
print X_test_enc_df.shape

(361500, 103)


In [356]:
selected_cols = [col for col in X_test_enc_df.columns if col not in excluded_cols_period]
X_test = X_test_enc_df[selected_cols].values
print X_test.shape

(361500, 71)


In [357]:
predictions = clf_xgb.predict_proba(X_test)[:,1]
print predictions

[ 0.4448325   0.58099896  0.52614629 ...,  0.5523901   0.63789403
  0.46348256]


In [358]:
print predictions.shape

(361500,)


In [350]:
print df_test.shape

(361500, 89)


In [359]:
submission = pd.DataFrame({'data_id': df_test['data_id'],'target': predictions})
submission.to_csv("submit_{:%Y%m%d-%H%M}.csv".format(datetime.datetime.now()), index=False)

## Prediction
Prepare train & test data for predition model

In [130]:
# Train all data, to train a subset
# selected_cols = [col for col in train1_df.columns if col not in ['period', 'target']]
selected_cols = [col for col in train1_df.columns if col not in excluded_cols]
X_train = df[selected_cols].values
y_train = df['target'].values
X_test = df_test[selected_cols].values
print 'Train data shape: {}'.format(X_train.shape)
print 'Train data shape: {}'.format(X_test.shape)

Train data shape: (560000, 57)
Train data shape: (361500, 57)


Fit all training data (above code is model with cross validation) and make prediction

In [131]:
chosen_est = clf_xgb
chosen_est.fit(X_train, y_train)
predictions = chosen_est.predict_proba(X_test)[:,1]
submission = pd.DataFrame({'data_id': df_test['data_id'],'target': predictions})
submission.to_csv("submit_{:%Y%m%d-%H%M}.csv".format(datetime.datetime.now()), index=False)